### **CÉLULA 1: IMPORTAÇÃO DE BIBLIOTECAS**

Nesta célula, importamos todas as bibliotecas necessárias para a análise e construção dos modelos de Machine Learning. Cada biblioteca tem uma função específica:

*   **`pandas` (pd)**: Utilizada para manipulação e análise de dados, especialmente DataFrames.
*   **`numpy` (np)**: Fundamental para operações numéricas e matemáticas de alto desempenho.
*   **`matplotlib.pyplot` (plt)** e **`seaborn` (sns)**: Usadas para visualização de dados e criação de gráficos estatísticos.
*   **`sklearn.model_selection.train_test_split`**: Para dividir os dados em conjuntos de treinamento e teste.
*   **`sklearn.preprocessing.LabelEncoder`** e **`StandardScaler`**: Para transformar variáveis categóricas em numéricas e para normalizar as features — etapa **obrigatória** para SVM e RNA, que são sensíveis à escala dos dados.
*   **`sklearn.svm.SVC`**: O algoritmo **Support Vector Machine** (Classificador).
*   **`sklearn.neural_network.MLPClassifier`**: O algoritmo de **Rede Neural Artificial** (Perceptron Multi-Camadas).
*   **`sklearn.metrics`**: Contém funções para avaliar o desempenho dos modelos (acurácia, precisão, recall, F1-Score e matriz de confusão).
*   **`sklearn.inspection.permutation_importance`**: Usada para calcular a importância das features de forma agnóstica ao modelo.

O `sns.set(style="whitegrid")` configura um estilo padrão para os gráficos.

In [ ]:
# ==============================================================================
# CÉLULA 1: IMPORTAÇÃO DE BIBLIOTECAS
# ==============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas de Machine Learning (Scikit-Learn)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, confusion_matrix, classification_report)
from sklearn.inspection import permutation_importance

# Configuração de estilo dos gráficos
sns.set(style="whitegrid")
print("Bibliotecas importadas com sucesso!")

### **CÉLULA 2: CARREGAMENTO E VISUALIZAÇÃO DOS DADOS**

Esta célula é responsável por carregar o conjunto de dados (`baseMLJurandir.csv`) em um DataFrame do pandas. Após o carregamento, exibimos as primeiras 5 linhas (`df.head()`) para ter uma visão inicial dos dados e confirmamos suas dimensões (`df.shape`).

Um bloco `try-except` é usado para lidar com a situação em que o arquivo não é encontrado, fornecendo uma mensagem de erro útil para o usuário.

In [ ]:
# ==============================================================================
# CÉLULA 2: CARREGAMENTO E VISUALIZAÇÃO DOS DADOS
# ==============================================================================
try:
    df = pd.read_csv('baseMLJurandir.csv')
    print(f"Dataset carregado com sucesso! Dimensões: {df.shape}")
    display(df.head())
except FileNotFoundError:
    print("ERRO: O arquivo 'baseMLJurandir.csv' não foi encontrado.")
    print("Por favor, faça o upload do arquivo na aba de arquivos à esquerda ou verifique o caminho.")

### **CÉLULA 3: PRÉ-PROCESSAMENTO E ENGENHARIA DE FEATURES**

Esta é uma etapa crucial para preparar os dados para o treinamento do modelo. Realizamos diversas transformações:

1.  **Correção de Colunas com Erro de Encoding**: Muitas vezes, caracteres especiais (como `ú` ou `ção`) podem ser carregados incorretamente (ex: `Ãºcleo`, `Ã§Ã£o`). Criamos uma função auxiliar (`corrigir_coluna`) para identificar e renomear essas colunas para seus nomes corretos (`núcleo`, `modulação`), garantindo que o código possa referenciá-las corretamente.

2.  **Criação da Variável Target (`aceito`)**: A coluna original `resultado` é usada para criar uma nova variável binária chamada `aceito`. Se `resultado` for `1`, `aceito` é `1` (indicando que o circuito foi aceito); caso contrário, `aceito` é `0` (indicando bloqueio). Esta será a variável que nossos modelos tentarão prever.

3.  **Remoção da Coluna Original `resultado`**: Após criar `aceito`, a coluna `resultado` se torna redundante e é removida do DataFrame.

4.  **Criação da Feature `slots_usados`**: Duas colunas existentes, `primeiro slot` e `ultimo slot`, são combinadas para criar uma nova e mais informativa feature, `slots_usados`, que representa a quantidade de slots utilizados. As colunas originais são então removidas.

5.  **Tratamento da Variável Categórica `modulação`**: A coluna `modulação` é uma variável categórica. Para que os algoritmos de Machine Learning possam processá-la, ela é convertida em um formato numérico usando `LabelEncoder`.

Ao final, exibimos a estrutura do DataFrame processado e suas colunas para verificar as mudanças.

In [ ]:
# ==============================================================================
# CÉLULA 3: PRÉ-PROCESSAMENTO E ENGENHARIA DE FEATURES
# ==============================================================================

print("Colunas do DataFrame antes do pré-processamento:", df.columns.tolist())

# --- Correção de Colunas com Erro de Encoding ---
def corrigir_coluna(df, prefixo_esperado, sufixos_erro, nome_correto):
    found_col = None
    for col in df.columns:
        if prefixo_esperado in col.lower():
            for sufixo in sufixos_erro:
                if sufixo in col:
                    found_col = col
                    break
            if found_col:
                break

    if found_col and found_col != nome_correto:
        df.rename(columns={found_col: nome_correto}, inplace=True)
        print(f"Coluna '{found_col}' renomeada para '{nome_correto}'.")
    elif found_col and found_col == nome_correto:
        print(f"Coluna '{nome_correto}' já está com o nome correto.")
    else:
        print(f"AVISO: A coluna '{nome_correto}' (ou sua variação com encoding errado) não foi encontrada no DataFrame.")

# Corrigir 'núcleo'
corrigir_coluna(df, 'n', ['Ãºcleo'], 'núcleo')

# Corrigir 'modulação'
corrigir_coluna(df, 'modula', ['Ã§Ã£o', 'ção'], 'modulação')
# --- Fim da Correção de Colunas ---

# 1. Criar variável target binária "aceito"
df['aceito'] = df['resultado'].apply(lambda x: 1 if x == 1 else 0)

# 2. Remover a coluna original "resultado"
df.drop('resultado', axis=1, inplace=True)

# 3. Criar feature "slots_usados" e remover "primeiro slot" e "ultimo slot"
df['slots_usados'] = df['ultimo slot'] - df['primeiro slot'] + 1
df.drop(['primeiro slot', 'ultimo slot'], axis=1, inplace=True)

# 4. Tratar variável categórica "modulação" com LabelEncoder
le = LabelEncoder()
if 'modulação' in df.columns and df['modulação'].dtype == 'object':
    df['modulação'] = le.fit_transform(df['modulação'])
    print("Classes de modulação codificadas:", list(le.classes_))
elif 'modulação' in df.columns:
    print("A coluna 'modulação' já parece ser numérica ou foi convertida, nenhuma alteração feita.")
else:
    print("ERRO: A coluna 'modulação' não foi encontrada no DataFrame após a tentativa de correção.")

print("\n--- Estrutura Final do Dataset ---")
display(df.head())
print(f"Colunas finais: {df.columns.tolist()}")

### **CÉLULA 4: SEPARAÇÃO EM TREINO E TESTE**

Para avaliar corretamente o desempenho de um modelo de Machine Learning, é fundamental dividirmos o conjunto de dados em duas partes:

*   **Conjunto de Treino (Training Set)**: Usado para treinar o modelo, ou seja, para que ele aprenda os padrões nos dados.
*   **Conjunto de Teste (Test Set)**: Usado para avaliar o desempenho do modelo em dados *nunca antes vistos*. Isso nos ajuda a ter uma ideia de como o modelo se comportará em situações reais e a evitar o *overfitting*.

Aqui, utilizamos `train_test_split` para separar os dados em **70% para treino** e **30% para teste**. O parâmetro `random_state=42` garante reprodutibilidade dos resultados.

In [ ]:
# ==============================================================================
# CÉLULA 4: SEPARAÇÃO EM TREINO E TESTE
# ==============================================================================

# Definir X (features) e y (target)
X = df.drop('aceito', axis=1)
y = df['aceito']

# Separar: 70% Treino, 30% Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

print(f"Tamanho do Treino: {X_train.shape[0]} amostras")
print(f"Tamanho do Teste:  {X_test.shape[0]} amostras")

### **CÉLULA 5: NORMALIZAÇÃO DAS FEATURES (StandardScaler)**

Esta etapa é **exclusiva e obrigatória** para os algoritmos SVM e RNA, sendo desnecessária no Random Forest (que é invariante à escala).

#### **Por que escalar os dados?**

*   **SVM**: O algoritmo busca o hiperplano de margem máxima entre as classes. Se as features tiverem escalas muito diferentes (ex: uma feature varia de 0 a 1000 e outra de 0 a 1), a que tem maior magnitude dominará o cálculo da distância, distorcendo completamente o hiperplano ótimo.

*   **RNA (MLPClassifier)**: A Rede Neural usa gradiente descendente para otimizar os pesos. Com features em escalas díspares, o gradiente se torna excessivamente assimétrico, causando convergência lenta ou instável.

#### **Como funciona o `StandardScaler`?**

Ele transforma cada feature para ter **média 0** e **desvio padrão 1** (padronização Z-score):

$$z = \frac{x - \mu}{\sigma}$$

**Regra de ouro**: O scaler é ajustado (`fit`) **apenas nos dados de treino** para evitar *data leakage*. Em seguida, é aplicado (`transform`) tanto no treino quanto no teste.

In [ ]:
# ==============================================================================
# CÉLULA 5: NORMALIZAÇÃO DAS FEATURES (StandardScaler)
# ==============================================================================

scaler = StandardScaler()

# IMPORTANTE: fit() apenas no treino para evitar data leakage
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("Normalização concluída com StandardScaler.")
print(f"Média das features (treino após scaling): {X_train_scaled.mean(axis=0).round(4)}")
print(f"Desvio padrão (treino após scaling):      {X_train_scaled.std(axis=0).round(4)}")

### **CÉLULA 6: TREINAMENTO DOS MODELOS**

Nesta célula, inicializamos e treinamos dois modelos de classificação:

1.  **Support Vector Machine (SVC)**: Algoritmo que busca o hiperplano de margem máxima que separa as classes no espaço de features. Utilizamos o **kernel RBF** (*Radial Basis Function*), que projeta os dados em um espaço de dimensão superior — permitindo separar classes que não são linearmente separáveis no espaço original. Os parâmetros principais são:
    *   `C=1.0` — parâmetro de regularização (penalidade por erros de classificação). Valores maiores = margem menor e menos erros no treino.
    *   `kernel='rbf'` — tipo de kernel utilizado.
    *   `gamma='scale'` — define o alcance de influência de cada amostra de treino.
    *   `probability=True` — habilita a estimativa de probabilidades (necessário para alguns contextos).

2.  **RNA — Rede Neural Artificial (MLPClassifier)**: Implementa um Perceptron Multi-Camadas com backpropagation. A arquitetura definida é:
    *   `hidden_layer_sizes=(100, 50)` — duas camadas ocultas com 100 e 50 neurônios, respectivamente.
    *   `activation='relu'` — função de ativação ReLU nas camadas ocultas.
    *   `solver='adam'` — otimizador Adam, eficiente para grandes conjuntos de dados.
    *   `max_iter=500` — número máximo de épocas de treinamento.
    *   `random_state=42` — garante reprodutibilidade.

Ambos os modelos são treinados (`.fit()`) usando os dados **normalizados** de treino (`X_train_scaled`, `y_train`).

In [ ]:
# ==============================================================================
# CÉLULA 6: TREINAMENTO DOS MODELOS
# ==============================================================================

# 1. Support Vector Machine (SVC com kernel RBF)
svm_model = SVC(C=1.0, kernel='rbf', gamma='scale', probability=True, random_state=42)
svm_model.fit(X_train_scaled, y_train)
print("Modelo SVM (kernel RBF) treinado.")
print(f"  Número de vetores de suporte por classe: {svm_model.n_support_}")

# 2. Rede Neural Artificial (MLP - Perceptron Multi-Camadas)
rna_model = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    n_iter_no_change=15
)
rna_model.fit(X_train_scaled, y_train)
print(f"\nModelo RNA (MLP) treinado.")
print(f"  Camadas: {rna_model.n_layers_} | Iterações realizadas: {rna_model.n_iter_}")

### **CÉLULA 7: AVALIAÇÃO DOS MODELOS**

Após treinar os modelos, avaliamos seu desempenho com uma função auxiliar `avaliar_modelo` que calcula métricas e exibe a matriz de confusão.

#### **Métricas de Avaliação:**

*   **Acurácia (Accuracy)**: Proporção de previsões corretas sobre o total.
*   **Precisão (Precision)**: Das previsões positivas, quantas estavam corretas? Minimiza **falsos positivos** (circuito bloqueado previsto como aceito).
*   **Recall (Sensibilidade)**: Dos casos realmente positivos, quantos foram identificados? Minimiza **falsos negativos** (circuito válido previsto como bloqueio). Um **alto Recall** para `aceito=1` significa que o modelo não bloqueia erroneamente circuitos válidos.
*   **F1-Score**: Média harmônica entre Precisão e Recall. Útil em datasets desbalanceados.

#### **Matriz de Confusão:**

Mostra a distribuição de:
*   **VP** — Previu Aceito, era Aceito.
*   **VN** — Previu Bloqueio, era Bloqueio.
*   **FP** — Previu Aceito, era Bloqueio (circuito bloqueado aceito erroneamente).
*   **FN** — Previu Bloqueio, era Aceito (circuito válido bloqueado erroneamente).

O `classification_report` fornece um resumo detalhado por classe.

In [ ]:
# ==============================================================================
# CÉLULA 7: AVALIAÇÃO DOS MODELOS
# ==============================================================================

def avaliar_modelo(modelo, nome_modelo, X_test_s, y_test):
    """Função auxiliar para calcular métricas e plotar matriz de confusão."""
    y_pred = modelo.predict(X_test_s)

    # Métricas
    acc  = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec  = recall_score(y_test, y_pred)
    f1   = f1_score(y_test, y_pred)

    print(f"--- Resultados: {nome_modelo} ---")
    print(f"Acurácia:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred))

    # Matriz de Confusão
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Matriz de Confusão — {nome_modelo}')
    plt.xlabel('Predito (0=Bloqueio, 1=Aceito)')
    plt.ylabel('Real (0=Bloqueio, 1=Aceito)')
    plt.tight_layout()
    plt.show()

    return [acc, prec, rec, f1]

# Avaliar SVM
metrics_svm = avaliar_modelo(svm_model, "SVM (kernel RBF)", X_test_scaled, y_test)

# Avaliar RNA
metrics_rna = avaliar_modelo(rna_model, "RNA (MLP)", X_test_scaled, y_test)

### **CÉLULA 8: COMPARAÇÃO FINAL DOS MODELOS**

Nesta célula, consolidamos as métricas de avaliação de ambos os modelos (SVM e RNA) em um DataFrame para facilitar a comparação lado a lado. Em seguida, visualizamos essas métricas com um gráfico de barras comparativo.

Essa comparação é fundamental para escolher o modelo mais adequado para implantação, levando em conta os objetivos do negócio — por exemplo, se é mais crítico minimizar falsos negativos (circuitos válidos bloqueados) ou falsos positivos (circuitos inválidos aceitos).

In [ ]:
# ==============================================================================
# CÉLULA 8: COMPARAÇÃO FINAL DOS MODELOS
# ==============================================================================

# Criar DataFrame comparativo
df_compare = pd.DataFrame({
    'Métrica':  ['Acurácia', 'Precision', 'Recall', 'F1-Score'],
    'SVM (RBF)': metrics_svm,
    'RNA (MLP)': metrics_rna
})

print("\n--- Comparação Lado a Lado ---")
display(df_compare)

# Gráfico de barras comparativo
df_compare_melted = df_compare.melt(id_vars="Métrica", var_name="Modelo", value_name="Valor")
plt.figure(figsize=(10, 6))
sns.barplot(x="Métrica", y="Valor", hue="Modelo", data=df_compare_melted, palette="magma")
plt.title("Comparação de Desempenho: SVM vs RNA (MLP)")
plt.ylim(0, 1.1)
plt.tight_layout()
plt.show()

### **CÉLULA 9: IMPORTÂNCIA DAS FEATURES (Permutation Importance)**

Diferentemente do Random Forest, que possui importância de features nativa (`feature_importances_`), o **SVM** e a **RNA não oferecem esse atributo diretamente**. Para contornar essa limitação, usamos a técnica de **Permutation Importance** (`sklearn.inspection.permutation_importance`).

#### **Como funciona a Permutation Importance?**

O método avalia a queda no desempenho do modelo quando os valores de uma feature são aleatoriamente permutados (embaralhados) no conjunto de teste:

1.  Calcula a métrica base do modelo (acurácia) no conjunto de teste original.
2.  Para cada feature, embaralha seus valores e calcula novamente a métrica.
3.  A **importância** é a diferença entre a métrica original e a métrica após o embaralhamento.

**Interpretação**: Se embaralhar uma feature causa grande queda na acurácia, essa feature é muito importante para o modelo. Se a queda for pequena (ou nula), a feature é pouco relevante.

Esta abordagem é **agnóstica ao modelo** — funciona igualmente para SVM, RNA, Random Forest ou qualquer outro algoritmo, tornando as comparações entre modelos mais justas.

O gráfico mostra os resultados para ambos os modelos lado a lado.

In [ ]:
# ==============================================================================
# CÉLULA 9: IMPORTÂNCIA DAS FEATURES (Permutation Importance)
# ==============================================================================

feature_names = X.columns.tolist()

def plotar_permutation_importance(modelo, nome_modelo, X_test_s, y_test, feature_names, color):
    result = permutation_importance(
        modelo, X_test_s, y_test,
        n_repeats=30,
        random_state=42,
        scoring='accuracy'
    )
    sorted_idx = result.importances_mean.argsort()[::-1]

    plt.figure(figsize=(10, 6))
    plt.barh(
        range(len(feature_names)),
        result.importances_mean[sorted_idx],
        xerr=result.importances_std[sorted_idx],
        align='center',
        color=color,
        alpha=0.85
    )
    plt.yticks(range(len(feature_names)), [feature_names[i] for i in sorted_idx])
    plt.gca().invert_yaxis()
    plt.xlabel("Queda média na acurácia (maior = mais importante)")
    plt.title(f"Permutation Importance — {nome_modelo}")
    plt.tight_layout()
    plt.show()

    df_imp = pd.DataFrame({
        'Feature': [feature_names[i] for i in sorted_idx],
        'Importância Média': result.importances_mean[sorted_idx].round(4),
        'Desvio Padrão':     result.importances_std[sorted_idx].round(4)
    })
    display(df_imp)

print("Calculando Permutation Importance para o SVM...")
plotar_permutation_importance(svm_model, "SVM (kernel RBF)", X_test_scaled, y_test, feature_names, color="steelblue")

print("\nCalculando Permutation Importance para a RNA (MLP)...")
plotar_permutation_importance(rna_model, "RNA (MLP)", X_test_scaled, y_test, feature_names, color="coral")